In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

In [38]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import accuracy_score
import nltk


In [14]:
data = pd.read_csv('spam.csv')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will 蘝 b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [15]:
data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1,inplace=True)
data.head()

,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [16]:
data.rename({'v1':'label','v2':'text'},axis=1,inplace=True)
data.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [17]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   5572 non-null   object
 1   text    5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [18]:
data.duplicated().sum()

409

In [19]:
data.drop_duplicates(inplace=True)

In [20]:
data.duplicated().sum()

0

In [21]:
data.reset_index(drop=True, inplace=True)

In [22]:
data.isnull().sum()

,0
label,0
text,0


In [1]:
def preprocess_text(text):
    # Remove special characters and numbers using regex
    text = re.sub(r'[^a-zA-Z]',' ',text)
    # Tokenization and lowercasing
    tokens = word_tokenize(text.lower())
    # Remove stopwords and lemmatize tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens if token not in stop_words]
    return tokens

In [26]:
import nltk
nltk.download('punkt_tab')
data['tokens'] = data['text'].apply(lambda x:preprocess_text(x))
data.head()

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


,label,text,tokens
0,ham,"Go until jurong point, crazy.. Available only ...","[go, until, jurong, point, ,, crazy, .., avail..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, ..., joking, wif, u, oni, ...]"
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,"[free, entry, in, 2, a, wkly, comp, to, win, f..."
3,ham,U dun say so early hor... U c already then say...,"[u, dun, say, so, early, hor, ..., u, c, alrea..."
4,ham,"Nah I don't think he goes to usf, he lives aro...","[nah, i, do, n't, think, he, goes, to, usf, ,,..."


In [29]:
# OneHotEncoding
y=pd.get_dummies(data.label)
y=y.iloc[:,1].values
y.shape

(5163,)

In [32]:
#Split the data into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(data['tokens'], y, test_size=0.2, random_state=42)

In [33]:
print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(4130,)
(1033,)
(4130,)
(1033,)


In [34]:
# Train Word2Vec model
word2vec_model = Word2Vec(sentences=x_train, vector_size=100, window=5, min_count=1, workers=4)

In [35]:
# Function to calculate average Word2Vec for a sentence
def average_word_vectors(tokens, model):
    vectors = [model.wv[word] for word in tokens if word in model.wv]
    if not vectors:
        return np.zeros(model.vector_size)
    return np.mean(vectors, axis=0)

In [36]:
# Create average vectors for training and testing sets
x_train_vectors = np.array([average_word_vectors(tokens, word2vec_model) for tokens in x_train])
x_test_vectors = np.array([average_word_vectors(tokens, word2vec_model) for tokens in x_test])

In [39]:
# Train a classifier (e.g., Naive Bayes)
classifier = BernoulliNB()
classifier.fit(x_train_vectors.tolist(), y_train)

BernoulliNB()

In [40]:
# Make predictions on the test set
y_pred = classifier.predict(x_test_vectors.tolist())

# Evaluate the model
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')

Accuracy: 0.85
